analyze split_data method from data_formatter class(GenericDataFormatter)

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import tensorflow as tf
import tensorflow.compat.v1 as tf1
from tensorflow.compat.v1 import Session, ConfigProto
from tensorflow.python.eager.context import PhysicalDevice
from typing import Dict, List, Union, Generator
import os
from numpy import load

In [2]:
import sys
sys.path.insert(0, '..')

In [11]:
from data_formatters.base import GenericDataFormatter, InputTypes, DataTypes
from data_formatters.sorgenia_wind import SorgeniaFormatter

In [4]:
from expt_settings.configs import ExperimentConfig
from libs.hyperparam_opt import HyperparamOptManager
from libs.tft_model import TemporalFusionTransformer
import libs.utils as utils

In [5]:
if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


In [6]:
gpu: List[PhysicalDevice] = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

In [7]:
# Tensorflow setup
default_keras_session: Session = tf1.keras.backend.get_session()
tf_config: ConfigProto = utils.get_default_tensorflow_config(tf_device="gpu", gpu_id=0)

Selecting GPU ID=0


In [8]:
file_path: str = r'C:\Users\Lorenzo\PycharmProjects\TFT\outputs\data\sorgenia_wind\data\sorgenia_wind\data\sorgenia_final.csv'

In [9]:
raw_data: DataFrame = pd.read_csv(file_path)

In [12]:
raw_data.head()

,plant_name_up,time,kwh,dew_point_2m_C,temperature_2m_C,msl_pressure_hPa,sfc_pressure_hPa,precipitation_1h_mm,wind_speed_mean_10m_1h_ms,wind_speed_mean_100m_1h_ms,...,days_from_start,id,hour,day,day_of_week,month,categorical_id,hours_from_start,categorical_day_of_week,categorical_hour
0,UP_MPNTLCDMRN_1,2016-12-31 23:00:00,225.000,-0.902831,4.980909,1028.935552,983.718765,0.000000e+00,1.381208,2.254668,...,0,UP_MPNTLCDMRN_1,23,31,5,12,UP_MPNTLCDMRN_1,0.0,5,23
1,UP_MPNTLCDMRN_1,2017-01-01 00:00:00,65.625,-0.783022,4.890773,1028.693079,983.524042,-1.734723e-15,1.278206,2.224647,...,0,UP_MPNTLCDMRN_1,0,1,6,1,UP_MPNTLCDMRN_1,1.0,6,0
2,UP_MPNTLCDMRN_1,2017-01-01 01:00:00,31.875,-0.565645,4.807044,1028.156679,983.034144,-1.734723e-15,1.211802,2.130597,...,0,UP_MPNTLCDMRN_1,1,1,6,1,UP_MPNTLCDMRN_1,2.0,6,1
3,UP_MPNTLCDMRN_1,2017-01-01 02:00:00,0.000,-0.331075,4.911959,1028.162779,983.062126,-1.734723e-15,1.017035,1.846413,...,0,UP_MPNTLCDMRN_1,2,1,6,1,UP_MPNTLCDMRN_1,3.0,6,2
4,UP_MPNTLCDMRN_1,2017-01-01 03:00:00,0.000,-0.668040,4.464566,1027.616920,982.551146,-1.734723e-15,0.715094,1.450024,...,0,UP_MPNTLCDMRN_1,3,1,6,1,UP_MPNTLCDMRN_1,4.0,6,3


In [13]:
config = ExperimentConfig('sorgenia_wind', r'C:\Users\Lorenzo\PycharmProjects\TFT\outputs')

In [14]:
formatter: SorgeniaFormatter = config.make_data_formatter()

# SPLIT THE DATA

In [15]:
train, valid, test = formatter.split_data(raw_data)

Setting scalers with training data...


In [16]:
train.head()

,plant_name_up,time,kwh,dew_point_2m_C,temperature_2m_C,msl_pressure_hPa,sfc_pressure_hPa,precipitation_1h_mm,wind_speed_mean_10m_1h_ms,wind_speed_mean_100m_1h_ms,...,days_from_start,id,hour,day,day_of_week,month,categorical_id,hours_from_start,categorical_day_of_week,categorical_hour
0,UP_MPNTLCDMRN_1,2016-12-31 23:00:00,-0.561958,-2.278493,-1.639455,2.258226,1.593520,-0.191301,-0.684613,-0.611277,...,0,UP_MPNTLCDMRN_1,1.661345,31,1.000944,12,0,-1.731980,5,23
1,UP_MPNTLCDMRN_1,2017-01-01 00:00:00,-0.688670,-2.255795,-1.651979,2.215322,1.574535,-0.191301,-0.746078,-0.622932,...,0,UP_MPNTLCDMRN_1,-1.661381,1,1.500937,1,0,-1.731839,6,0
2,UP_MPNTLCDMRN_1,2017-01-01 01:00:00,-0.715503,-2.214612,-1.663613,2.120409,1.526770,-0.191301,-0.785703,-0.659444,...,0,UP_MPNTLCDMRN_1,-1.516914,1,1.500937,1,0,-1.731697,6,1
3,UP_MPNTLCDMRN_1,2017-01-01 02:00:00,-0.740845,-2.170171,-1.649036,2.121488,1.529499,-0.191301,-0.901926,-0.769771,...,0,UP_MPNTLCDMRN_1,-1.372448,1,1.500937,1,0,-1.731556,6,2
4,UP_MPNTLCDMRN_1,2017-01-01 03:00:00,-0.740845,-2.234011,-1.711200,2.024902,1.479679,-0.191301,-1.082103,-0.923659,...,0,UP_MPNTLCDMRN_1,-1.227982,1,1.500937,1,0,-1.731415,6,3


looks like the data_formatter scaled "power_usage", "hour", "day_of_week", "hours_from_start"

In [17]:
column_definitions = formatter.get_column_definition()

In [19]:
train_samples, valid_samples = formatter.get_num_samples_for_calibration()

In [ ]:
# Sets up default params
fixed_params: Dict = formatter.get_experiment_params()
params: Dict = formatter.get_default_model_params()
params["model_folder"]: str = os.path.join(config.model_folder, "fixed")
model_folder = os.path.join(config.model_folder, "fixed")

In [ ]:
# Sets up hyperparam manager
print("*** Loading hyperparm manager ***")
opt_manager = HyperparamOptManager({k: [params[k]] for k in params},
                                   fixed_params, model_folder)

In [ ]:
model_folder: str = opt_manager.hyperparam_folder

# ANALYZE PARAMETERS

In [ ]:
fixed_params.keys()

In [ ]:
#column definition
fixed_params.get('column_definition')

In [ ]:
fixed_params.get('known_categorical_inputs')

# TRAINING

In [ ]:
print("*** Running calibration ***")
print("Params Selected:")
for k in params:
    print("{}: {}".format(k, params[k]))
    
best_loss = np.Inf
for _ in range(1):
    tf1.reset_default_graph()
    with tf.Graph().as_default(), tf1.Session(config=tf_config) as sess:
        tf1.keras.backend.set_session(sess)
        
        params: Dict = opt_manager.get_next_parameters()
        model: TemporalFusionTransformer = TemporalFusionTransformer(params, use_cudnn=False)

        if not os.path.exists(os.path.join(model.data_folder, 'data.npy')) and not model.training_data_cached():
            model.cache_batched_data(train, "train", num_samples=train_samples)
            model.cache_batched_data(valid, "valid", num_samples=valid_samples)
            
        sess.run(tf1.global_variables_initializer())
        model.fit() 
        
        val_loss: Series = model.evaluate()
            
        if val_loss < best_loss:
            opt_manager.update_score(params, val_loss, model)
            best_loss = val_loss
        tf1.keras.backend.set_session(default_keras_session)

In [ ]:
model_ckpt: str = r'C:\Users\Lorenzo\PycharmProjects\TFT\outputs\saved_models\electricity\fixed\TemporalFusionTransformer.check'

In [ ]:
folder: str = r'C:\Users\Lorenzo\PycharmProjects\TFT\outputs\data\electricity\data\electricity'
train_size: int = 450000

In [ ]:
def traindata_gen(folder: str, minibatch_size: int) -> Generator:
    data: memmap = load(os.path.join(folder, 'data.npy'), mmap_mode='r')
    print(type(data))
    labels: memmap = load(os.path.join(folder, 'labels.npy'), mmap_mode='r')
    active_flags: memmap = load(os.path.join(folder, 'active_flags.npy'), mmap_mode='r')
    train_size: Tuple = data.shape
    for i in range(train_size[0] // minibatch_size + 1):
        upper = min((i + 1) * minibatch_size, train_size[0])

        yield data[i * minibatch_size:upper], np.concatenate([labels[i * minibatch_size:upper],
                                                                           labels[i * minibatch_size:upper],
                                                                           labels[i * minibatch_size:upper]], axis=-1), \
                      active_flags[i * minibatch_size:upper]

In [ ]:
num_batches = 0
for data, labels, active_flags in traindata_gen(folder, 64):
    num_batches += 1
num_batches

In [ ]:
import math
math.ceil(50000/64)